# Spotify API

Follows the exact same steps as the other Spotify API file. Please refer to the other file for the documentation of how we utilized this API.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext() 

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id = "ed85fecdaeeb475ebe9a9f54a40eb054",client_secret='8001bae67076404bbb1a5980f2db94b1')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Pandas

In [3]:
songs = pd.read_csv("cleaned_5_archit.csv")

In [4]:
songs.head()

,Unnamed: 0,genre,lyrics,song,artist
0,309023,Hip-Hop,I'm silent B.G. and you know me from thuging I...,silent-b-g,b-g
1,353899,Hip-Hop,Ah shit! Yeah Everywhere I go 'Nough gunshot a...,it-s-going-down,daz-dillinger
2,71018,Hip-Hop,Hound doggin this muthufucka Raise up off my n...,hound-dogs,blaze-ya-dead-homie
3,17996,Hip-Hop,"My Beamer sit on Jimmy Choos, damn My bitch, I...",jimmy-choo,fetty-wap
4,313382,Hip-Hop,I don't think you understand That what your do...,wasting-my-time,eminem


In [5]:
songs['song_artist'] = songs['song'] + ":" + songs['artist']
songs.head()

,Unnamed: 0,genre,lyrics,song,artist,song_artist
0,309023,Hip-Hop,I'm silent B.G. and you know me from thuging I...,silent-b-g,b-g,silent-b-g:b-g
1,353899,Hip-Hop,Ah shit! Yeah Everywhere I go 'Nough gunshot a...,it-s-going-down,daz-dillinger,it-s-going-down:daz-dillinger
2,71018,Hip-Hop,Hound doggin this muthufucka Raise up off my n...,hound-dogs,blaze-ya-dead-homie,hound-dogs:blaze-ya-dead-homie
3,17996,Hip-Hop,"My Beamer sit on Jimmy Choos, damn My bitch, I...",jimmy-choo,fetty-wap,jimmy-choo:fetty-wap
4,313382,Hip-Hop,I don't think you understand That what your do...,wasting-my-time,eminem,wasting-my-time:eminem


In [6]:
def clean_artist(artist):
    artist.replace('-', ' ')
    return artist.upper()

In [7]:
def find_song(song_artist):
    song_name, artist = song_artist.split(":")
    artist = clean_artist(artist)
    results = sp.search(q='track:' + song_name, type='track')
    lenTracks = len(results['tracks']['items'])
    if lenTracks == 0:
        return None
    idS = None
    for i in range(lenTracks):
        if results['tracks']['items'][i]['album']['artists'][0]['name'].upper() == artist:
            idS = results['tracks']['items'][i]['id']
    return idS     
# newresults = sp.audio_features(idS)
# print(newresults)


In [8]:
songIDS = songs['song_artist'].map(lambda x: find_song(x)).dropna()

retrying ...1secs
retrying ...1secs
retrying ...1secs


In [9]:
audio_feat = songIDS.map(lambda x: sp.audio_features(x))

In [10]:
series = audio_feat.map(lambda x: x[0])

In [94]:
df = pd.DataFrame(series.tolist())
df.T

,0,1,2,3,4,5,6,7,8,9,...,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894
acousticness,0.0107,0.701,0.402,0.175,0.0457,0.0476,0.042,0.0142,0.196,0.0376,...,0.123,0.000456,1.98e-05,0.0436,3.96e-05,2.01e-05,0.917,2.42e-06,0.000591,3.02e-05
analysis_url,https://api.spotify.com/v1/audio-analysis/4bQS...,https://api.spotify.com/v1/audio-analysis/1to4...,https://api.spotify.com/v1/audio-analysis/0coo...,https://api.spotify.com/v1/audio-analysis/6yqE...,https://api.spotify.com/v1/audio-analysis/0OIl...,https://api.spotify.com/v1/audio-analysis/1wOl...,https://api.spotify.com/v1/audio-analysis/4nVH...,https://api.spotify.com/v1/audio-analysis/3e3W...,https://api.spotify.com/v1/audio-analysis/3qVO...,https://api.spotify.com/v1/audio-analysis/2Y9E...,...,https://api.spotify.com/v1/audio-analysis/3YRC...,https://api.spotify.com/v1/audio-analysis/3IAZ...,https://api.spotify.com/v1/audio-analysis/0ABs...,https://api.spotify.com/v1/audio-analysis/2vII...,https://api.spotify.com/v1/audio-analysis/6ykC...,https://api.spotify.com/v1/audio-analysis/3kES...,https://api.spotify.com/v1/audio-analysis/1dnf...,https://api.spotify.com/v1/audio-analysis/7gj2...,https://api.spotify.com/v1/audio-analysis/3PjD...,https://api.spotify.com/v1/audio-analysis/5l8s...
danceability,0.852,0.522,0.899,0.689,0.73,0.539,0.884,0.754,0.803,0.656,...,0.228,0.17,0.352,0.334,0.353,0.374,0.169,0.174,0.312,0.291
duration_ms,318960,310637,212827,124872,257741,224347,302343,67227,208293,91653,...,189373,203147,228613,210013,276747,215693,131173,234893,205027,191720
energy,0.633,0.654,0.655,0.304,0.726,0.968,0.493,0.411,0.67,0.879,...,0.955,0.99,0.971,0.409,0.987,0.945,0.142,0.877,0.948,0.902
id,4bQSX3sgwmyGyxBon1MM1f,1to4zsRWRjzbRGcBSvEQsm,0cooaE59CnjLSyasgiAnQf,6yqEiIRmynE0DMPMHmONY5,0OIlfAyiH2LrOtOPRW0JUL,1wOl5O8Y725UaWazvJmCDL,4nVHpSgnqxniryRIozwmBQ,3e3W9kBiGUbPq3a7yJx9Dr,3qVO6LmdmwFogL3NSarFNj,2Y9EpUgeXzpL9dMNTXXKGU,...,3YRC58IcUs7Yw3nJ6hhcuB,3IAZKshkAnTND0rIn5dRY9,0ABszyc6ZwiWILesGKXNnZ,2vIIqADbDlxeKCzm3SEWAb,6ykCDNaR2S29FYuTZIc0qZ,3kES5rt6tHHVRISdZAEng7,1dnfZpzPzt37eQ9vQu7lHi,7gj2JtNUY4qbJHZP8T7yLK,3PjD03Mq7klIKAAv2R5GVi,5l8sPBqIz0qH88CTMtLyLV
instrumentalness,0,0,0,0.113,0,0,0,0,0,2.38e-05,...,0.829,0.832,0.633,0.855,0.00312,0.00302,0.939,0.239,8.63e-05,0.749
key,7,11,0,6,7,9,2,6,1,2,...,3,6,6,3,11,6,5,11,1,7
liveness,0.343,0.207,0.0879,0.194,0.0492,0.348,0.387,0.413,0.218,0.166,...,0.327,0.0965,0.094,0.103,0.219,0.346,0.112,0.0805,0.2,0.359
loudness,-5.122,-7.14,-5.073,-14.299,-3.18,-2.17,-13.294,-17.694,-6.934,-5.379,...,-9.556,-4.852,-4.571,-12.422,-4.042,-6.488,-16.13,-8.456,-5.451,-6.417


In [12]:
df.to_csv("archit_categories.csv")

In [78]:
# new_series = df['id']
# # songs_names = new_series.map(lambda x: )
# song_names = []
# for i in new_series:
#     song_names.append(sp.track(i)['name'])

In [86]:
songNames = songIDS.map(lambda x: (x, sp.track(x)))

In [95]:
songNames_2 = songNames.map(lambda x: (x[0], x[1]['name']))
dfNames = pd.DataFrame(songNames_2.tolist())
dfNames = dfNames.set_index(0)
dfNames = dfNames.rename(index = str, columns = {1: "Song Name"})
dfNames.head()

,Song Name
0,
4bQSX3sgwmyGyxBon1MM1f,She Say She Loves Me (feat. 8 Ball & Bun B)
1to4zsRWRjzbRGcBSvEQsm,The Book of Soul
0cooaE59CnjLSyasgiAnQf,Hate U 2
6yqEiIRmynE0DMPMHmONY5,TermsAndConditions
0OIlfAyiH2LrOtOPRW0JUL,BK Dance


In [96]:
dfCombined = df.join(dfNames, on='id', how = 'inner')

In [97]:
dfCombined.T

,0,1,2,3,4,5,6,7,8,9,...,2885,2886,2887,2888,2889,2890,2891,2892,2893,2894
acousticness,0.0107,0.701,0.402,0.175,0.0457,0.0476,0.042,0.0142,0.196,0.0376,...,0.123,0.000456,1.98e-05,0.0436,3.96e-05,2.01e-05,0.917,2.42e-06,0.000591,3.02e-05
analysis_url,https://api.spotify.com/v1/audio-analysis/4bQS...,https://api.spotify.com/v1/audio-analysis/1to4...,https://api.spotify.com/v1/audio-analysis/0coo...,https://api.spotify.com/v1/audio-analysis/6yqE...,https://api.spotify.com/v1/audio-analysis/0OIl...,https://api.spotify.com/v1/audio-analysis/1wOl...,https://api.spotify.com/v1/audio-analysis/4nVH...,https://api.spotify.com/v1/audio-analysis/3e3W...,https://api.spotify.com/v1/audio-analysis/3qVO...,https://api.spotify.com/v1/audio-analysis/2Y9E...,...,https://api.spotify.com/v1/audio-analysis/3YRC...,https://api.spotify.com/v1/audio-analysis/3IAZ...,https://api.spotify.com/v1/audio-analysis/0ABs...,https://api.spotify.com/v1/audio-analysis/2vII...,https://api.spotify.com/v1/audio-analysis/6ykC...,https://api.spotify.com/v1/audio-analysis/3kES...,https://api.spotify.com/v1/audio-analysis/1dnf...,https://api.spotify.com/v1/audio-analysis/7gj2...,https://api.spotify.com/v1/audio-analysis/3PjD...,https://api.spotify.com/v1/audio-analysis/5l8s...
danceability,0.852,0.522,0.899,0.689,0.73,0.539,0.884,0.754,0.803,0.656,...,0.228,0.17,0.352,0.334,0.353,0.374,0.169,0.174,0.312,0.291
duration_ms,318960,310637,212827,124872,257741,224347,302343,67227,208293,91653,...,189373,203147,228613,210013,276747,215693,131173,234893,205027,191720
energy,0.633,0.654,0.655,0.304,0.726,0.968,0.493,0.411,0.67,0.879,...,0.955,0.99,0.971,0.409,0.987,0.945,0.142,0.877,0.948,0.902
id,4bQSX3sgwmyGyxBon1MM1f,1to4zsRWRjzbRGcBSvEQsm,0cooaE59CnjLSyasgiAnQf,6yqEiIRmynE0DMPMHmONY5,0OIlfAyiH2LrOtOPRW0JUL,1wOl5O8Y725UaWazvJmCDL,4nVHpSgnqxniryRIozwmBQ,3e3W9kBiGUbPq3a7yJx9Dr,3qVO6LmdmwFogL3NSarFNj,2Y9EpUgeXzpL9dMNTXXKGU,...,3YRC58IcUs7Yw3nJ6hhcuB,3IAZKshkAnTND0rIn5dRY9,0ABszyc6ZwiWILesGKXNnZ,2vIIqADbDlxeKCzm3SEWAb,6ykCDNaR2S29FYuTZIc0qZ,3kES5rt6tHHVRISdZAEng7,1dnfZpzPzt37eQ9vQu7lHi,7gj2JtNUY4qbJHZP8T7yLK,3PjD03Mq7klIKAAv2R5GVi,5l8sPBqIz0qH88CTMtLyLV
instrumentalness,0,0,0,0.113,0,0,0,0,0,2.38e-05,...,0.829,0.832,0.633,0.855,0.00312,0.00302,0.939,0.239,8.63e-05,0.749
key,7,11,0,6,7,9,2,6,1,2,...,3,6,6,3,11,6,5,11,1,7
liveness,0.343,0.207,0.0879,0.194,0.0492,0.348,0.387,0.413,0.218,0.166,...,0.327,0.0965,0.094,0.103,0.219,0.346,0.112,0.0805,0.2,0.359
loudness,-5.122,-7.14,-5.073,-14.299,-3.18,-2.17,-13.294,-17.694,-6.934,-5.379,...,-9.556,-4.852,-4.571,-12.422,-4.042,-6.488,-16.13,-8.456,-5.451,-6.417


In [98]:
def cap_songs(song):
    parts = song.split("-")
    for i in range(len(parts)):
        parts[i] = parts[i].capitalize()
    
    return " ".join(parts)

In [99]:
dfCombined = dfCombined.set_index('Song Name')

In [100]:
dfCombined.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
Song Name,,,,,,,,,,,,,,,,,,
She Say She Loves Me (feat. 8 Ball & Bun B),0.0107,https://api.spotify.com/v1/audio-analysis/4bQS...,0.852,318960,0.633,4bQSX3sgwmyGyxBon1MM1f,0.000,7,0.3430,-5.122,1,0.1990,121.974,4,https://api.spotify.com/v1/tracks/4bQSX3sgwmyG...,audio_features,spotify:track:4bQSX3sgwmyGyxBon1MM1f,0.749
The Book of Soul,0.7010,https://api.spotify.com/v1/audio-analysis/1to4...,0.522,310637,0.654,1to4zsRWRjzbRGcBSvEQsm,0.000,11,0.2070,-7.140,0,0.3220,77.740,3,https://api.spotify.com/v1/tracks/1to4zsRWRjzb...,audio_features,spotify:track:1to4zsRWRjzbRGcBSvEQsm,0.508
Hate U 2,0.4020,https://api.spotify.com/v1/audio-analysis/0coo...,0.899,212827,0.655,0cooaE59CnjLSyasgiAnQf,0.000,0,0.0879,-5.073,1,0.3310,93.032,4,https://api.spotify.com/v1/tracks/0cooaE59CnjL...,audio_features,spotify:track:0cooaE59CnjLSyasgiAnQf,0.494
TermsAndConditions,0.1750,https://api.spotify.com/v1/audio-analysis/6yqE...,0.689,124872,0.304,6yqEiIRmynE0DMPMHmONY5,0.113,6,0.1940,-14.299,1,0.0329,110.030,4,https://api.spotify.com/v1/tracks/6yqEiIRmynE0...,audio_features,spotify:track:6yqEiIRmynE0DMPMHmONY5,0.156
BK Dance,0.0457,https://api.spotify.com/v1/audio-analysis/0OIl...,0.730,257741,0.726,0OIlfAyiH2LrOtOPRW0JUL,0.000,7,0.0492,-3.180,1,0.3350,185.831,4,https://api.spotify.com/v1/tracks/0OIlfAyiH2Lr...,audio_features,spotify:track:0OIlfAyiH2LrOtOPRW0JUL,0.560


In [101]:
songs['song'] = songs['song'].map(lambda x: cap_songs(x))

In [102]:
songs.head()

,Unnamed: 0,genre,lyrics,song,artist,song_artist
0,309023,Hip-Hop,I'm silent B.G. and you know me from thuging I...,Silent B G,b-g,silent-b-g:b-g
1,353899,Hip-Hop,Ah shit! Yeah Everywhere I go 'Nough gunshot a...,It S Going Down,daz-dillinger,it-s-going-down:daz-dillinger
2,71018,Hip-Hop,Hound doggin this muthufucka Raise up off my n...,Hound Dogs,blaze-ya-dead-homie,hound-dogs:blaze-ya-dead-homie
3,17996,Hip-Hop,"My Beamer sit on Jimmy Choos, damn My bitch, I...",Jimmy Choo,fetty-wap,jimmy-choo:fetty-wap
4,313382,Hip-Hop,I don't think you understand That what your do...,Wasting My Time,eminem,wasting-my-time:eminem


In [103]:
songs = songs.set_index('song')
songs.head()

,Unnamed: 0,genre,lyrics,artist,song_artist
song,,,,,
Silent B G,309023,Hip-Hop,I'm silent B.G. and you know me from thuging I...,b-g,silent-b-g:b-g
It S Going Down,353899,Hip-Hop,Ah shit! Yeah Everywhere I go 'Nough gunshot a...,daz-dillinger,it-s-going-down:daz-dillinger
Hound Dogs,71018,Hip-Hop,Hound doggin this muthufucka Raise up off my n...,blaze-ya-dead-homie,hound-dogs:blaze-ya-dead-homie
Jimmy Choo,17996,Hip-Hop,"My Beamer sit on Jimmy Choos, damn My bitch, I...",fetty-wap,jimmy-choo:fetty-wap
Wasting My Time,313382,Hip-Hop,I don't think you understand That what your do...,eminem,wasting-my-time:eminem


In [104]:
dfFinal = dfCombined.join(songs, how="inner")

In [105]:
dfFinal['genre'].value_counts()

Metal      1113
Hip-Hop     546
Name: genre, dtype: int64

In [106]:
dfFinal.T

,10 Million People,13 Candles,1597,24 Pills,29th Lobotomy,3 Foot Tall,30 Seconds,30072003,4 Poisons 3 Words,40 Water,...,You Had Too Much To Drink,You Must Be Crazy,Young Black Male,Young Madonna,Your Rotting Face,Yuri Barbarossa,Zakliatie,Zombie Walk,Zombification,Zos Kia Cultus
acousticness,0.118,0.000538,0.0122,9.89e-05,1.39e-05,0.25,0.934,0.579,0.000117,0.0613,...,0.00942,0.0909,0.144,0.00896,2.29e-05,0.367,0.00791,0.42,1.98e-05,7.81e-05
analysis_url,https://api.spotify.com/v1/audio-analysis/4x1s...,https://api.spotify.com/v1/audio-analysis/1kQR...,https://api.spotify.com/v1/audio-analysis/5T8B...,https://api.spotify.com/v1/audio-analysis/7vvl...,https://api.spotify.com/v1/audio-analysis/5xLr...,https://api.spotify.com/v1/audio-analysis/4EkX...,https://api.spotify.com/v1/audio-analysis/0ypg...,https://api.spotify.com/v1/audio-analysis/3ZoP...,https://api.spotify.com/v1/audio-analysis/66YX...,https://api.spotify.com/v1/audio-analysis/6gGi...,...,https://api.spotify.com/v1/audio-analysis/3jAW...,https://api.spotify.com/v1/audio-analysis/1o5X...,https://api.spotify.com/v1/audio-analysis/5hNO...,https://api.spotify.com/v1/audio-analysis/3ba1...,https://api.spotify.com/v1/audio-analysis/3D49...,https://api.spotify.com/v1/audio-analysis/6xEl...,https://api.spotify.com/v1/audio-analysis/19Wb...,https://api.spotify.com/v1/audio-analysis/6LLx...,https://api.spotify.com/v1/audio-analysis/2DGg...,https://api.spotify.com/v1/audio-analysis/783l...
danceability,0.734,0.326,0.601,0.401,0.219,0.609,0.499,0.413,0.29,0.852,...,0.805,0.753,0.725,0.643,0.303,0.876,0.494,0.462,0.154,0.321
duration_ms,262773,314240,184867,226507,102307,220947,39320,251280,176920,217880,...,440533,294347,155000,243281,236413,228333,318187,208010,207600,332573
energy,0.825,0.872,0.705,0.967,0.682,0.742,0.227,0.575,0.972,0.549,...,0.41,0.889,0.41,0.886,0.803,0.851,0.863,0.83,0.821,0.954
id,4x1sTfUXhTPeNFAzq0iZrF,1kQRDEOvVOqbvrwpmXj7kN,5T8BVmTesssOfzA7ljYD2x,7vvlqRgdi4jqwzLVJPhPmX,5xLrvOuRGO3Lg8Lud5jHfN,4EkXHwwBPQvd8mEdP3wOVB,0ypgRXDPoSkz6SR0K192T8,3ZoPiZNPJWzbrIQPmrl6HC,66YXL5ah0oyDIYJHcYSU9A,6gGi5c6qEP7lGYdpbqyajb,...,3jAWDfWxvdHHOoY0qI9xxc,1o5XOhumqrNSq9dQRuFWiv,5hNOICzhequtUXSM0N7jPB,3ba14W63KoZZGoEmx05K3B,3D49TUMHZrMVHtoR6hhrML,6xElmWb2gCpccrYsfmjSvA,19WbFwOThXuy5lgR0ku2me,6LLx4ryHkTWxwX9SRBWCOd,2DGgrFIQ8FP0wtLkaT1B6v,783lzX2BhYhm4kRuQqdKcO
instrumentalness,0,0.73,0,0.0638,0.599,0,0.94,0.531,0.0114,0,...,0.00027,0,9.34e-06,0,0.336,1.47e-06,9.67e-06,7.23e-06,0.478,0.00142
key,5,9,11,7,7,11,3,8,11,1,...,1,11,2,1,2,11,2,5,6,10
liveness,0.101,0.2,0.13,0.0768,0.349,0.641,0.116,0.209,0.581,0.0521,...,0.093,0.0943,0.456,0.125,0.0781,0.079,0.75,0.126,0.266,0.163
loudness,-6.355,-9.643,-10.488,-2.732,-9.169,-5.739,-20.263,-8.275,-3.299,-6.484,...,-11.054,-5.03,-16.943,-4.577,-8.051,-3.355,-4.084,-2.605,-10.243,-4.915


In [107]:
dfFinal.to_csv("additional_features_metal_hip_archit.csv")